In [1]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

%matplotlib widget

In [2]:
# cluster plotting function

def plot_clusters(band_num):
    
    plt.figure(figsize=(10,3))
    
#     mean vs sigma
    plt.subplot(1, 3, 1)
    plt.scatter(orig_mu_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_mu_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Mean")
    plt.ylabel("Variance")
    plt.legend()
    
#     sigma vs beta
    plt.subplot(1, 3, 2)
    plt.scatter(orig_beta_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_beta_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Beta")
    plt.ylabel("Variance")
    plt.legend()
     
#     beta vs mean
    plt.subplot(1, 3, 3)
    plt.scatter(orig_mu_feat[:,band_num], orig_beta_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_mu_feat[:,band_num], laser_beta_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Mean")
    plt.ylabel("Beta")
    plt.legend()
    
    plt.suptitle("Laser vs Original for {}th Band".format(band_num))
    
    fig = plt.figure(figsize=(10,3))
#     plt.figure(figsize=(20,6))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(orig_mu_feat[:,band_num], orig_sigma_feat[:,band_num], orig_beta_feat[:,band_num], marker='o', label="Original")
    ax.scatter(laser_mu_feat[:,band_num], laser_sigma_feat[:,band_num], laser_beta_feat[:,band_num], marker='x', label="Laser")
    ax.set_xlabel('Mean')
    ax.set_ylabel('Variance')
    ax.set_zlabel('Beta')
    ax.legend()

In [3]:
pywt.families(short=False)

['Haar',
 'Daubechies',
 'Symlets',
 'Coiflets',
 'Biorthogonal',
 'Reverse biorthogonal',
 'Discrete Meyer (FIR Approximation)',
 'Gaussian',
 'Mexican hat wavelet',
 'Morlet wavelet',
 'Complex Gaussian wavelets',
 'Shannon wavelets',
 'Frequency B-Spline wavelets',
 'Complex Morlet wavelets']

In [4]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files.sort()
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_009.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_010.mp3']
10


In [5]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files.sort()
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_006.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_009.mp3']
10


In [6]:
audio_data, sr = librosa.load(orig_files[0], res_type='kaiser_fast')
len(audio_data)

550839

In [7]:
audio_data

array([-7.9890615e-06, -1.6427866e-05, -1.5079420e-05, ...,
       -7.6267117e-04, -8.6591946e-04,  0.0000000e+00], dtype=float32)

In [8]:
wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

In [9]:
wp.data
print(len(wp.data))

550839


In [10]:
wp

In [11]:
print(wp.maxlevel)

5


In [12]:
print(len(wp['a'].data))
wp['a'].data

275420


array([-1.7265374e-05, -2.3079712e-05, -3.1976459e-05, ...,
       -1.4990689e-03, -1.1515875e-03,  0.0000000e+00], dtype=float32)

In [13]:
print([node.path for node in wp.get_level(1, 'freq')])

['a', 'd']


In [14]:
print([node.path for node in wp.get_level(2, 'freq')])

['aa', 'ad', 'dd', 'da']


In [15]:
print([node.path for node in wp.get_level(3, 'freq')])

['aaa', 'aad', 'add', 'ada', 'dda', 'ddd', 'dad', 'daa']


In [16]:
print([node.path for node in wp.get_level(4, 'freq')])

['aaaa', 'aaad', 'aadd', 'aada', 'adda', 'addd', 'adad', 'adaa', 'ddaa', 'ddad', 'dddd', 'ddda', 'dada', 'dadd', 'daad', 'daaa']


In [17]:
print([node.path for node in wp.get_level(5, 'freq')])

['aaaaa', 'aaaad', 'aaadd', 'aaada', 'aadda', 'aaddd', 'aadad', 'aadaa', 'addaa', 'addad', 'adddd', 'addda', 'adada', 'adadd', 'adaad', 'adaaa', 'ddaaa', 'ddaad', 'ddadd', 'ddada', 'dddda', 'ddddd', 'dddad', 'dddaa', 'dadaa', 'dadad', 'daddd', 'dadda', 'daada', 'daadd', 'daaad', 'daaaa']


In [18]:
db1 = pywt.Wavelet('db1')

In [19]:
coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)

In [20]:
print(np.shape(coeffs))
coeffs

(6,)


/home/hashim/pyenvs/laser_injection/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


[array([-8.442947e-05, -9.586333e-05, -8.271822e-05, ..., -3.198916e-03,
        -4.245463e-03, -3.419802e-03], dtype=float32),
 array([-1.5706100e-06, -3.7424070e-06, -4.5164415e-06, ...,
         1.0395040e-03,  8.5276016e-04,  5.5678596e-04], dtype=float32),
 array([-1.4038218e-05,  2.5063691e-06, -3.2384232e-06, ...,
        -5.0764717e-04,  7.9983729e-05,  0.0000000e+00], dtype=float32),
 array([ 1.25814749e-05, -9.48029992e-07, -3.56083729e-06, ...,
        -1.15168106e-04,  2.30781734e-05, -8.36706779e-04], dtype=float32),
 array([ 4.1113572e-06,  1.0996355e-06, -5.7519628e-07, ...,
         6.0782826e-04,  1.2242608e-04, -8.1429532e-04], dtype=float32),
 array([ 5.9671361e-06,  1.7541925e-06, -7.2170587e-07, ...,
         2.1305401e-05,  7.3007541e-05,  0.0000000e+00], dtype=float32)]

In [21]:
np.shape(coeffs[0])

(17214,)

In [22]:
np.shape(coeffs[1])

(17214,)

In [23]:
np.shape(coeffs[2])

(34428,)

In [24]:
np.shape(coeffs[3])

(68855,)

In [25]:
np.shape(coeffs[-1])

(275420,)

In [26]:
x = [3, 7, 1, 1, -2, 5, 4, 6]

In [27]:
# coeffs = pywt.wavedec(x, db1)
# coeffs

# Wavelet Analysis starts from here

In [28]:
%%capture
# compute coefficients for both laser and original
laser_spec = []

laser_mu_feat = []
laser_sigma_feat = []
laser_beta_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for cf in coeffs:
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]
    
#         print(f_param)
#         print(f.summary())

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    laser_mu_feat.append(mu_feat)
    laser_sigma_feat.append(sigma_feat)
    laser_beta_feat.append(beta_feat)    
    
laser_mu_feat = np.array(laser_mu_feat)
laser_sigma_feat = np.array(laser_sigma_feat)
laser_beta_feat = np.array(laser_beta_feat)

#     laser_spec.append(S_db)

# laser_spec = np.array(laser_spec)
# laser_spec

In [29]:
print(laser_mu_feat.shape)
print(laser_sigma_feat.shape)
print(laser_beta_feat.shape)

(10, 6)
(10, 6)
(10, 6)


In [30]:
%%capture
orig_spec = []

orig_mu_feat = []
orig_sigma_feat = []
orig_beta_feat = []
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    

    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for cf in coeffs:
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    orig_mu_feat.append(mu_feat)
    orig_sigma_feat.append(sigma_feat)
    orig_beta_feat.append(beta_feat)
    
orig_mu_feat = np.array(orig_mu_feat)
orig_sigma_feat = np.array(orig_sigma_feat)
orig_beta_feat = np.array(orig_beta_feat)


In [31]:
print(orig_mu_feat.shape)
print(orig_sigma_feat.shape)
print(orig_beta_feat.shape)

(10, 6)
(10, 6)
(10, 6)


In [32]:
aprox_detail_coeff = 0 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
aprox_detail_coeff = 1 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
aprox_detail_coeff = 2 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
aprox_detail_coeff = 3 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
aprox_detail_coeff = 4 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
aprox_detail_coeff = 5 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#  Below Results are for the VAD audios 

In [38]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files.sort()
# print(laser_files)
print(len(laser_files))

72


In [39]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original_cleaned/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files.sort()
# print(orig_files)
print(len(orig_files))

74


In [40]:
%%capture
# compute spectograms for both laser and original
laser_spec = []

laser_mu_feat = []
laser_sigma_feat = []
laser_beta_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for cf in coeffs:
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]
    
#         print(f_param)
#         print(f.summary())

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    laser_mu_feat.append(mu_feat)
    laser_sigma_feat.append(sigma_feat)
    laser_beta_feat.append(beta_feat)    
    
laser_mu_feat = np.array(laser_mu_feat)
laser_sigma_feat = np.array(laser_sigma_feat)
laser_beta_feat = np.array(laser_beta_feat)

In [41]:
print(laser_mu_feat.shape)
print(laser_sigma_feat.shape)
print(laser_beta_feat.shape)

(72, 6)
(72, 6)
(72, 6)


In [42]:
%%capture
orig_spec = []

orig_mu_feat = []
orig_sigma_feat = []
orig_beta_feat = []
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    

    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for cf in coeffs:
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    orig_mu_feat.append(mu_feat)
    orig_sigma_feat.append(sigma_feat)
    orig_beta_feat.append(beta_feat)
    
orig_mu_feat = np.array(orig_mu_feat)
orig_sigma_feat = np.array(orig_sigma_feat)
orig_beta_feat = np.array(orig_beta_feat)


In [43]:
print(orig_mu_feat.shape)
print(orig_sigma_feat.shape)
print(orig_beta_feat.shape)

(74, 6)
(74, 6)
(74, 6)


In [44]:
aprox_detail_coeff = 0 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
aprox_detail_coeff = 1 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
aprox_detail_coeff = 2 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
aprox_detail_coeff = 3 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
aprox_detail_coeff = 4 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
aprox_detail_coeff = 5 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# SVM with Linear Kernel

In [50]:
import sklearn

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

In [51]:
X_laser = np.concatenate((laser_mu_feat, laser_sigma_feat, laser_beta_feat), axis=1)
laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape

['laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser']
(72,)


(72, 18)

In [52]:
X_orig = np.concatenate((orig_mu_feat, orig_sigma_feat, orig_beta_feat), axis=1)
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

['original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original']
(74,)


(74, 18)

In [53]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [54]:
X = np.concatenate((X_laser, X_orig))
X.shape

(146, 18)

In [55]:
y = np.concatenate((laser_label, orig_label))
y.shape

(146,)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [57]:
X_train.shape

(97, 18)

In [58]:
y_train.shape

(97,)

In [59]:
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC())])

In [60]:
y_pred = clf.predict(X_test)

In [61]:
print("accuracy = ", accuracy_score(y_test, y_pred))

accuracy =  0.9591836734693877


In [62]:
cm = confusion_matrix(y_test, y_pred)

In [63]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [64]:
(tn, fp, fn, tp)

(25, 1, 1, 22)

In [65]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# svm with non-linear kernal

In [66]:
clf_svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [67]:
clf_svc.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [68]:
y_pred_svc = clf_svc.predict(X_test)

In [69]:
print("accuracy = ", accuracy_score(y_test, y_pred_svc))

accuracy =  0.9387755102040817


In [70]:
cm_svc = confusion_matrix(y_test, y_pred_svc)

In [71]:
tn, fp, fn, tp = cm_svc.ravel()

In [72]:
(tn, fp, fn, tp)

(24, 2, 1, 22)

In [73]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_svc, display_labels=clf_svc.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Histograms of Coefficients

In [74]:
# laser histograms of approximation coefficients
nrows = int(len(laser_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    ax[i].hist(coeffs[0], bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
# Original histograms of Approximation coefficients
nrows = int(len(orig_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    ax[i].hist(coeffs[0], bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
# laser histograms of Detail coefficients
nrows = int(len(laser_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    ax[i].hist(coeffs[1], bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
# Original histograms of Detail coefficients
nrows = int(len(orig_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    ax[i].hist(coeffs[1], bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
# laser histograms of Detail coefficients
nrows = int(len(laser_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    ax[i].hist(coeffs[-1], bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
# Original histograms of Detail coefficients
nrows = int(len(orig_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    ax[i].hist(coeffs[-1], bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …